In [ ]:
# Databricks notebook source
# ==================================================================================
# 🚀 PRODUCTION PROMOTION SCRIPT — SIMPLIFIED VERSION
# ==================================================================================
# Logic: If model passes UAT → Directly promote to Production
# No metric comparison needed (already done in UAT evaluation)
# ==================================================================================

import mlflow
from mlflow.tracking import MlflowClient
import os
import time
import sys
import requests
from pyspark.sql import SparkSession

print("=" * 80)
print("🚀 PRODUCTION PROMOTION STARTED")
print("=" * 80)

# ==================================================================================
# ✅ CONFIGURATION
# ==================================================================================
UC_CATALOG = "workspace"
UC_SCHEMA = "ml"
MODEL_NAME = f"{UC_CATALOG}.{UC_SCHEMA}.house_price_xgboost_uc2"

PRODUCTION_ALIAS = "production"
STAGING_ALIAS = "Staging"

# UAT results table (to check if model passed UAT)
UAT_RESULTS_TABLE = "workspace.default.uat_inference_house_price_xgboost"

# ==================================================================================
# ✅ SLACK NOTIFICATION SETUP
# ==================================================================================
def get_slack_webhook():
    """Retrieve Slack webhook URL from available scopes"""
    for scope in ["shared-scope", "dev-scope"]:
        try:
            webhook = dbutils.secrets.get(scope, "SLACK_WEBHOOK_URL")
            if webhook and webhook.strip():
                print(f"✓ Slack webhook configured from scope '{scope}'")
                return webhook
        except Exception as e:
            print(f"⚠️ Slack webhook not found in scope '{scope}': {e}")
    print("⚠️ No Slack webhook configured")
    return None

SLACK_WEBHOOK_URL = get_slack_webhook()

def send_slack_notification(message, level="info"):
    """Send Slack message"""
    if not SLACK_WEBHOOK_URL:
        print(f"⚠️ Slack webhook not configured")
        print(f"📢 Message: {message}")
        return

    emoji_map = {"info": "ℹ️", "success": "✅", "warning": "⚠️", "error": "❌"}
    formatted_message = f"{emoji_map.get(level, 'ℹ️')} {message}"

    try:
        response = requests.post(SLACK_WEBHOOK_URL, json={"text": formatted_message}, timeout=5)
        if response.status_code == 200:
            print(f"📢 Slack notification sent successfully")
        else:
            print(f"⚠️ Slack notification failed: {response.status_code}")
    except Exception as e:
        print(f"⚠️ Slack notification error: {e}")

# ==================================================================================
# ✅ INITIALIZATION
# ==================================================================================
try:
    if "DATABRICKS_RUNTIME_VERSION" in os.environ:
        mlflow.set_registry_uri("databricks-uc")
        print("✅ MLflow connected to Unity Catalog")
    client = MlflowClient()
    spark = SparkSession.builder.appName("Production_Promotion").getOrCreate()
except Exception as e:
    print(f"❌ Initialization failed: {e}")
    send_slack_notification(f"❌ Production promotion failed: Initialization error", "error")
    raise e

# ==================================================================================
# ✅ HELPER: Wait until model is READY
# ==================================================================================
def wait_until_ready(client, model_name, version, timeout=300):
    """Wait for model version to become READY"""
    start = time.time()
    while time.time() - start < timeout:
        mv = client.get_model_version(model_name, version)
        status = mv.status
        
        if status == "READY":
            print(f"✅ Model v{version} is READY")
            return True
        elif status == "FAILED_REGISTRATION":
            print(f"❌ Model v{version} registration failed")
            return False
        
        print(f"⏳ Model v{version} status: {status}")
        time.sleep(5)
    
    print(f"⏰ Timeout: Model v{version} not ready")
    return False

# ==================================================================================
# ✅ STEP 1: Get Staging Model
# ==================================================================================
def get_staging_version(client):
    """Find model with @Staging alias"""
    print(f"\n{'='*80}")
    print(f"📋 STEP 1: Finding STAGING Model")
    print(f"{'='*80}")
    
    print(f"🔍 Model: {MODEL_NAME}")
    print(f"🔍 Looking for: @{STAGING_ALIAS}")
    
    # Method 1: Direct alias lookup
    try:
        print(f"\n🔄 Trying direct alias lookup...")
        staging_mv = client.get_model_version_by_alias(MODEL_NAME, STAGING_ALIAS)
        
        print(f"✅ Staging model found!")
        print(f"   Version: v{staging_mv.version}")
        print(f"   Run ID: {staging_mv.run_id}")
        print(f"   Status: {staging_mv.status}")
        
        return staging_mv
        
    except Exception as e:
        print(f"⚠️ Direct lookup failed: {e}")
    
    # Method 2: Search all versions
    try:
        print(f"\n🔄 Searching all versions...")
        versions = client.search_model_versions(f"name='{MODEL_NAME}'")
        
        if not versions:
            print(f"❌ No versions found for: {MODEL_NAME}")
            return None
        
        print(f"📊 Found {len(versions)} total version(s)")
        print(f"\n📋 Available versions:")
        
        staging_versions = []
        for v in versions:
            mv = client.get_model_version(MODEL_NAME, v.version)
            aliases = mv.aliases if mv.aliases else []
            print(f"   v{v.version}: Aliases={aliases}, Status={mv.status}")
            
            if any(alias.lower() == STAGING_ALIAS.lower() for alias in aliases):
                staging_versions.append(mv)
        
        if not staging_versions:
            print(f"\n❌ No version with @{STAGING_ALIAS} alias found")
            return None
        
        staging_mv = max(staging_versions, key=lambda x: int(x.version))
        print(f"\n✅ Selected staging version: v{staging_mv.version}")
        
        return staging_mv
        
    except Exception as e:
        print(f"❌ Version search failed: {e}")
        import traceback
        traceback.print_exc()
        return None

# ==================================================================================
# ✅ STEP 2: Check UAT Status
# ==================================================================================
def check_uat_status(staging_version):
    """
    Check if staging model passed UAT
    Returns: (passed: bool, uat_metrics: dict)
    """
    print(f"\n{'='*80}")
    print(f"📋 STEP 2: Checking UAT Status")
    print(f"{'='*80}")
    
    try:
        print(f"🔍 Reading UAT results from: {UAT_RESULTS_TABLE}")
        uat_df = spark.table(UAT_RESULTS_TABLE).toPandas()
        
        if uat_df.empty:
            print(f"⚠️ No UAT results found")
            return False, None
        
        print(f"📊 Found {len(uat_df)} UAT result(s)")
        
        # Filter for this specific model version
        version_results = uat_df[uat_df['model_version'] == int(staging_version)]
        
        if version_results.empty:
            print(f"\n⚠️ No UAT results for version v{staging_version}")
            print(f"💡 Run UAT inference script first!")
            return False, None
        
        # Get latest UAT result for this version
        latest_result = version_results.sort_values('timestamp', ascending=False).iloc[0]
        
        uat_status = latest_result['uat_status']
        
        print(f"\n📊 UAT Results for v{staging_version}:")
        print(f"   Timestamp: {latest_result['timestamp']}")
        print(f"   UAT Status: {uat_status}")
        print(f"   MAE:  {latest_result['mae']:,.2f}")
        print(f"   RMSE: {latest_result['rmse']:,.2f}")
        print(f"   R²:   {latest_result['r2']:.4f}")
        print(f"   MAPE: {latest_result['mape']:.2f}%")
        
        uat_metrics = {
            'mae': float(latest_result['mae']),
            'rmse': float(latest_result['rmse']),
            'r2': float(latest_result['r2']),
            'mape': float(latest_result['mape'])
        }
        
        if uat_status == "PASSED":
            print(f"\n✅ Model v{staging_version} PASSED UAT")
            return True, uat_metrics
        else:
            print(f"\n❌ Model v{staging_version} FAILED UAT")
            return False, uat_metrics
            
    except Exception as e:
        print(f"❌ Failed to check UAT status: {e}")
        import traceback
        traceback.print_exc()
        return False, None

# ==================================================================================
# ✅ STEP 3: Promote to Production
# ==================================================================================
def promote_to_production(client, staging_mv, uat_metrics):
    """Promote staging model to production"""
    print(f"\n{'='*80}")
    print(f"📋 STEP 3: Promoting to PRODUCTION")
    print(f"{'='*80}")
    
    version = staging_mv.version
    
    # Wait for model to be ready
    print(f"\n⏳ Ensuring model v{version} is READY...")
    if not wait_until_ready(client, MODEL_NAME, version):
        msg = f"❌ Model v{version} not ready for promotion"
        print(msg)
        send_slack_notification(msg, "error")
        return False
    
    # Promote to production
    try:
        print(f"\n🚀 Setting @{PRODUCTION_ALIAS} alias to v{version}...")
        
        client.set_registered_model_alias(
            name=MODEL_NAME,
            alias=PRODUCTION_ALIAS,
            version=version
        )
        
        print(f"\n{'='*80}")
        print(f"✅✅ PROMOTION SUCCESSFUL ✅✅")
        print(f"{'='*80}")
        print(f"   Model: {MODEL_NAME}")
        print(f"   Version: v{version}")
        print(f"   Promoted: @{STAGING_ALIAS} → @{PRODUCTION_ALIAS}")
        print(f"   Run ID: {staging_mv.run_id}")
        
        if uat_metrics:
            print(f"\n📊 UAT Metrics:")
            print(f"   RMSE: {uat_metrics['rmse']:,.2f}")
            print(f"   MAPE: {uat_metrics['mape']:.2f}%")
            print(f"   R²:   {uat_metrics['r2']:.4f}")
        
        print(f"{'='*80}")
        
        # Send success notification
        metrics_text = ""
        if uat_metrics:
            metrics_text = (
                f"\n📊 Performance Metrics:\n"
                f"   • RMSE: {uat_metrics['rmse']:,.2f}\n"
                f"   • MAPE: {uat_metrics['mape']:.2f}%\n"
                f"   • R²: {uat_metrics['r2']:.4f}"
            )
        
        send_slack_notification(
            f"🎉 *PRODUCTION DEPLOYMENT SUCCESS!*\n\n"
            f"Model: `{MODEL_NAME}`\n"
            f"Version: *v{version}*\n"
            f"Status: @{STAGING_ALIAS} → @{PRODUCTION_ALIAS}\n"
            f"Run ID: {staging_mv.run_id[:8]}..."
            f"{metrics_text}\n\n"
            f"🚀 Model is now LIVE in production!",
            "success"
        )
        
        return True
        
    except Exception as e:
        msg = f"❌ Failed to promote: {e}"
        print(msg)
        send_slack_notification(msg, "error")
        import traceback
        traceback.print_exc()
        return False

# ==================================================================================
# ✅ MAIN EXECUTION
# ==================================================================================
def main():
    """Main production promotion pipeline"""
    
    try:
        # Step 1: Get staging model
        staging_mv = get_staging_version(client)
        
        if not staging_mv:
            error_msg = (
                f"❌ *No staging model found*\n\n"
                f"Model: `{MODEL_NAME}`\n"
                f"Expected alias: @{STAGING_ALIAS}\n\n"
                f"💡 *Next steps:*\n"
                f"1. Run Model Registration script\n"
                f"2. Run Model Evaluation (UAT Staging Promotion)\n"
                f"3. Verify alias is set to '@{STAGING_ALIAS}'"
            )
            print(f"\n{error_msg}")
            send_slack_notification(error_msg, "error")
            sys.exit(1)
        
        # Step 2: Check UAT status
        uat_passed, uat_metrics = check_uat_status(staging_mv.version)
        
        if not uat_passed:
            warning_msg = (
                f"⚠️ *Model NOT promoted to production*\n\n"
                f"Model: `{MODEL_NAME}`\n"
                f"Version: v{staging_mv.version}\n"
                f"Reason: UAT not passed or results not found\n\n"
                f"💡 *Next steps:*\n"
                f"1. Run UAT Inference script\n"
                f"2. Ensure model passes UAT validation\n"
                f"3. Re-run production promotion"
            )
            print(f"\n{warning_msg}")
            send_slack_notification(warning_msg, "warning")
            sys.exit(0)
        
        # Step 3: Promote to production
        success = promote_to_production(client, staging_mv, uat_metrics)
        
        if success:
            print(f"\n✨ Production promotion completed successfully!")
            sys.exit(0)
        else:
            print(f"\n❌ Production promotion failed")
            sys.exit(1)
            
    except Exception as e:
        error_msg = f"❌ Production promotion script failed: {str(e)}"
        print(f"\n{error_msg}")
        send_slack_notification(error_msg, "error")
        import traceback
        traceback.print_exc()
        sys.exit(1)

# ==================================================================================
# ✅ RUN
# ==================================================================================
if __name__ == "__main__":
    main()





# # Databricks notebook source
# # ==================================================================================
# # 🚀 PRODUCTION PROMOTION SCRIPT — CLEAN VERSION (Single Model Architecture)
# # ==================================================================================

# import mlflow
# from mlflow.tracking import MlflowClient
# import os
# import time
# import sys

# print("=" * 80)
# print("🚀 PRODUCTION PROMOTION STARTED")
# print("=" * 80)

# # ==================================================================================
# # ✅ CONFIGURATION (Fixed model name — MUST match training + registration)
# # ==================================================================================
# UC_CATALOG = "workspace"
# UC_SCHEMA = "ml"
# MODEL_NAME = f"{UC_CATALOG}.{UC_SCHEMA}.house_price_xgboost_uc2"

# PRODUCTION_ALIAS = "production"
# STAGING_ALIAS = "staging"

# METRIC_KEY = "test_rmse"
# TOL = 1e-6  # threshold to treat metrics as identical


# # ==================================================================================
# # ✅ MLflow Initialization (Unity Catalog)
# # ==================================================================================
# try:
#     if "DATABRICKS_RUNTIME_VERSION" in os.environ:
#         mlflow.set_registry_uri("databricks-uc")
#         print("✅ MLflow connected to Unity Catalog")
#     client = MlflowClient()
# except Exception as e:
#     print(f"❌ MLflow client creation failed: {e}")
#     raise e


# # ==================================================================================
# # ✅ Helper: Wait until model version is READY
# # ==================================================================================
# def wait_until_ready(client, model_name, version, timeout=300):
#     start = time.time()
#     while time.time() - start < timeout:
#         mv = client.get_model_version(model_name, version)
#         if mv.status == "READY":
#             return True
#         if mv.status == "FAILED_REGISTRATION":
#             print("❌ Registration failed")
#             return False
#         time.sleep(5)
#     print("⏰ Timeout: model not ready")
#     return False


# # ==================================================================================
# # ✅ Helper: get metric of a run
# # ==================================================================================
# def get_metric(client, run_id):
#     try:
#         run = client.get_run(run_id)
#         return run.data.metrics.get(METRIC_KEY, None)
#     except:
#         return None


# # ==================================================================================
# # ✅ STEP 1: Pick latest staging version
# # ==================================================================================
# def get_staging_version(client):
#     versions = client.search_model_versions(f"name='{MODEL_NAME}'")

#     staging_versions = []
#     for v in versions:
#         mv = client.get_model_version(MODEL_NAME, v.version)
#         if STAGING_ALIAS in mv.aliases:
#             staging_versions.append(mv)

#     if not staging_versions:
#         print("❌ No staging model found")
#         return None

#     staging_version = max(staging_versions, key=lambda x: int(x.version))
#     print(f"✅ Staging Version Found: v{staging_version.version}")
#     return staging_version


# # ==================================================================================
# # ✅ STEP 2: Pick current production version (if any)
# # ==================================================================================
# def get_prod_version(client):
#     versions = client.search_model_versions(f"name='{MODEL_NAME}'")

#     for v in versions:
#         mv = client.get_model_version(MODEL_NAME, v.version)
#         if PRODUCTION_ALIAS in mv.aliases:
#             print(f"✅ Current Production Version: v{mv.version}")
#             return mv

#     print("ℹ️ No production model exists yet")
#     return None


# # ==================================================================================
# # ✅ STEP 3: Compare metrics (RMSE) and decide promotion
# # ==================================================================================
# def should_promote(new_rmse, old_rmse):
#     if old_rmse is None:
#         print("🟢 No production model → Promote Staging to Production")
#         return True

#     print(f"\n📊 Metric Comparison")
#     print(f"   New (Staging) RMSE: {new_rmse}")
#     print(f"   Old (Production) RMSE: {old_rmse}")

#     if new_rmse < old_rmse - TOL:
#         print("🟢 New staging model is better → Promote")
#         return True
#     else:
#         print("⛔ Staging model is NOT better → No promotion")
#         return False


# # ==================================================================================
# # ✅ STEP 4: Promote Staging → Production
# # ==================================================================================
# def promote_to_production(client, version):
#     print(f"\n⏳ Waiting for v{version} to become READY...")
#     if not wait_until_ready(client, MODEL_NAME, version):
#         print("❌ Model not ready for promotion")
#         return False

#     client.set_registered_model_alias(
#         name=MODEL_NAME,
#         alias=PRODUCTION_ALIAS,
#         version=version
#     )

#     print(f"✅✅ SUCCESS: Promoted Staging v{version} → PRODUCTION")
#     return True


# # ==================================================================================
# # ✅ MAIN EXECUTION
# # ==================================================================================
# if __name__ == "__main__":
#     staging_mv = get_staging_version(client)
#     if not staging_mv:
#         sys.exit(1)

#     prod_mv = get_prod_version(client)

#     # Fetch metrics
#     new_rmse = get_metric(client, staging_mv.run_id)
#     old_rmse = get_metric(client, prod_mv.run_id) if prod_mv else None

#     # Decide
#     if should_promote(new_rmse, old_rmse):
#         promote_to_production(client, staging_mv.version)
#     else:
#         print("\n✅ Production model remains unchanged.")
